In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.5 MB/s eta 0:00:00


In [ ]:
!pip install -U transformers==4.41.2 peft==0.10.0 accelerate==0.29.3 datasets evaluate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 70.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.2
    Uninstalling tokenizers-0.22.2:
      Successfully uninstalled tokenizers-0.22.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.6
    Uninstalling transfor

In [ ]:
import transformers, peft, accelerate
print(transformers.__version__)
print(peft.__version__)
print(accelerate.__version__)


4.41.2
0.10.0
0.29.3


In [ ]:
from datasets import load_dataset,DatasetDict,Dataset

from transformers import (
    AutoTokenizer,
    AutoConfig,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)

from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
import evaluate
import torch
import numpy as np

In [ ]:
dataset = load_dataset("lex_glue", "ecthr_a")
print(dataset)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

ecthr_a/train-00000-of-00001.parquet:   0%|          | 0.00/42.4M [00:00<?, ?B/s]

ecthr_a/test-00000-of-00001.parquet:   0%|          | 0.00/5.68M [00:00<?, ?B/s]

ecthr_a/validation-00000-of-00001.parque(…):   0%|          | 0.00/5.26M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 9000
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 1000
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 1000
    })
})


In [ ]:
model_checkpoint = "roberta-base"

tokenizer = AutoTokenizer.from_pretrained(
    model_checkpoint,
    add_prefix_space = True
)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({"pad_token":"[PAD]"})

In [ ]:
def tokenize_function(examples):
    # 1. Join paragraphs into a single document
    texts = [" ".join(doc) for doc in examples["text"]]

    tokenizer.truncation_side = "left"
    tokenized = tokenizer(
        texts,
        truncation=True,
        max_length=512
    )

    # 2. Convert label indices to multi-hot FLOAT vectors
    num_labels = 10
    multi_hot_labels = []

    for label_list in examples["labels"]:
        label_vector = [0.0] * num_labels   # 👈 FLOAT, not int
        for idx in label_list:
            label_vector[idx] = 1.0
        multi_hot_labels.append(label_vector)

    tokenized["labels"] = multi_hot_labels
    return tokenized


In [ ]:
#Done for better outputs
id2label = {
    0: "Article_2_Right_to_Life",
    1: "Article_3_Prohibition_of_Torture",
    2: "Article_5_Right_to_Liberty",
    3: "Article_6_Right_to_Fair_Trial",
    4: "Article_8_Private_and_Family_Life",
    5: "Article_9_Freedom_of_Thought",
    6: "Article_10_Freedom_of_Expression",
    7: "Article_11_Freedom_of_Assembly",
    8: "Article_14_Non_Discrimination",
    9: "Protocol1_Article1_Property"
}

label2id = {v: k for k, v in id2label.items()}

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,num_labels = 10,id2label = id2label,
                                           label2id = label2id,problem_type="multi_label_classification")

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.resize_token_embeddings(len(tokenizer))

Embedding(50265, 768, padding_idx=1)

In [ ]:
tokenized_dataset = dataset.map(tokenize_function,batched = True,remove_columns=["text"] )

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

class MultiLabelDataCollator(DataCollatorWithPadding):
    def __call__(self, features):
        batch = super().__call__(features)
        batch["labels"] = batch["labels"].float()  # 👈 FORCE FLOAT
        return batch


In [ ]:
data_collator = MultiLabelDataCollator(tokenizer)


In [ ]:
batch = data_collator([tokenized_dataset["train"][0]])
print(batch["labels"])
print(batch["labels"].dtype)


tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])
torch.float32


In [ ]:
peft_config = LoraConfig(
    task_type="SEQ_CLS",
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["query", "value"]
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


trainable params: 893,194 || all params: 125,546,516 || trainable%: 0.7114446728254888


In [ ]:
from sklearn.metrics import f1_score


In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred

    # Convert logits → binary predictions (multi-label)
    preds = (logits > 0).astype(int)

    return {
        "micro_f1": f1_score(labels, preds, average="micro"),
        "macro_f1": f1_score(labels, preds, average="macro"),
    }


In [ ]:
training_args = TrainingArguments(
    output_dir="legal-lora-multilabel",
    learning_rate=1e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=50,
    fp16=torch.cuda.is_available()
)


/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,   # 👈 IMPORTANT
    compute_metrics=compute_metrics,
)


/usr/local/lib/python3.12/dist-packages/accelerate/accelerator.py:469: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Micro F1,Macro F1
1,0.120800,0.170784,0.679516,0.579540
2,0.132200,0.162119,0.687619,0.572554
3,0.147200,0.155413,0.694311,0.582114
4,0.111900,0.155479,0.701788,0.599425
5,0.106000,0.154994,0.706558,0.610302


/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in ver

TrainOutput(global_step=5625, training_loss=0.12528335439893934, metrics={'train_runtime': 1611.8867, 'train_samples_per_second': 27.918, 'train_steps_per_second': 3.49, 'total_flos': 1.196432308224e+16, 'train_loss': 0.12528335439893934, 'epoch': 5.0})

In [ ]:
trainer.evaluate(tokenized_dataset["test"])


{'eval_loss': 0.15992076694965363,
 'eval_micro_f1': 0.6791907514450867,
 'eval_macro_f1': 0.541326440509142,
 'eval_runtime': 13.3101,
 'eval_samples_per_second': 75.131,
 'eval_steps_per_second': 9.391,
 'epoch': 5.0}